In [ ]:
import os
import zarr
import s3fs
import sunpy.map

import dask.array as da
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import sunpy.visualization.colormaps as cm

from astropy.time import Time
from dask.distributed import Client, LocalCluster
from sunpy.visualization import axis_labels_from_ctype, wcsaxes_compat

from matplotlib import animation
from IPython.display import HTML


def get_sdo_solar_images_from_aws(wanted_times, sav_folder_path):

    



    return fetched_images_path

    

In [ ]:
# this problem is still a work in progress
# DASK is not freeing up memory after the computation is done
# will need to address this issue in the future
# fix dask cache size

from dask.cache import Cache
cache = Cache(1e9)  # Leverage 1 GB of memory
cache.register()    # Turn cache on globally

# AWS path to the solar images data (for 2015) on AWS S3
AWS_ZARR_ROOT = (
    "s3://gov-nasa-hdrl-data1/contrib/fdl-sdoml/fdl-sdoml-v2/sdomlv2.zarr/2015/"
)

# Function to connect to AWS S3 to access the zarr data in S3
def s3_connection(path_to_zarr: os.path) -> s3fs.S3Map:
    """
    Instantiate connection to aws for a given path `path_to_zarr`
    """
    return s3fs.S3Map(
        root=path_to_zarr,
        s3=s3fs.S3FileSystem(anon=True),
        # anonymous access requires no credentials
        check=False,
    )


def load_single_aws_zarr(
    path_to_zarr: os.path,
    cache_max_single_size: int = None,
) -> Union[zarr.Array, zarr.Group]:
    """
    load zarr from s3 using LRU cache
    """
    return zarr.open(
        zarr.LRUStoreCache(
            store=s3_connection(path_to_zarr),
            max_size=cache_max_single_size,
        ),
        mode="r",
    )

# first, we create a group with the store data located on GCP.
root = load_single_aws_zarr(
    path_to_zarr=AWS_ZARR_ROOT,
)